In [ ]:
import numpy as np
import cv2
import os
from tqdm.notebook import tqdm

In [ ]:
# either using it from the colorize module or using it directly
# from colorize.colorize import doColorize

# This applied to semanticKITTI dataset only
def load_calib(calib_file):
    calib = {}
    with open(calib_file) as f:
        for line in f:
            key, *vals = line.strip().split()
            calib[key] = np.array(vals, dtype=np.float32).reshape(-1)
    return calib

def doColorize(pc_file, im_file, calib_file, cam: int = 0):
    # 1. Load point cloud
    points = np.fromfile(pc_file, dtype=np.float32).reshape(-1, 4) # (x, y, z, intensity)
    pts_velo = points[:, :3] # (x, y, z)
    # 2. Load image
    img = cv2.imread(im_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w, _ = img.shape
    # 3. Load calibration
    calib = load_calib(calib_file)
    if cam == 0:
        P = calib['P2:'].reshape(3, 4) # Left camera proj
    elif cam == 1:
        P = calib['P3:'].reshape(3, 4) # right camera proj
    Tr = calib['Tr:'].reshape(3, 4) # LiDAR to cam0
    # 4. Transform LiDAR points to camera frame
    pts_hom = np.hstack((pts_velo, np.ones((pts_velo.shape[0], 1))))  # N×4
    pts_cam = ((Tr @ pts_hom.T).reshape(3, -1)).T  # N×3 
    # 5. Project to image plane
    pts_img = (P @ np.hstack((pts_cam, np.ones((pts_cam.shape[0], 1)))).T).T
    pts_img[:, 0] /= pts_img[:, 2]
    pts_img[:, 1] /= pts_img[:, 2]
    # 6. Filter valid points
    valid = (pts_cam[:, 2] > 0) & \
            (pts_img[:, 0] >= 0) & (pts_img[:, 0] < w) & \
            (pts_img[:, 1] >= 0) & (pts_img[:, 1] < h)
    pts_valid = pts_velo[valid]
    uv = pts_img[valid, :2].astype(int)
    colors = img[uv[:, 1], uv[:, 0]] / 255.0  # RGB normalized
    # 7. Print number of valid points along with number of points
    # print(f'Number of valid points/ Number of points: {np.sum(valid)}/{pts_velo.shape[0]} ({np.sum(valid)/pts_velo.shape[0]*100:.2f}%)')
    # 8. Cascade the valid array to points
    points_add = np.concatenate((points, np.zeros((points.shape[0], 4))), axis=1)
    points_add[:, 4] = valid.astype(np.int32)  # Assign 1 if valid, 0 otherwise 
    points_add[valid, 5:8] = colors  # RGB
    # points_add is the original point cloud with additional columns for valid flag and RGB colors (x,y,z,intensity,flag,r,g,b)
    # check if points_add meet the expected dimention
    assert points_add.shape[1] == 8, f"Expected points_add to have 8 columns, but got {points_add.shape[1]}"
    # pts_valid is the (x, y, z) of the valid points
    # colors is the RGB values of the valid points

    return points_add, pts_valid, colors

In [ ]:
# either using it from projection module or using it directly

# This applied to semanticKITTI dataset only
proj_w=2048
proj_h=64
def doProjection(pointcloud: np.ndarray, wrap_around: int = 0):
    # get depth of all points
    depth = np.linalg.norm(pointcloud[:, :3], 2, axis=1)
    # get point cloud components
    x = pointcloud[:, 0]
    y = pointcloud[:, 1]

    # get angles of all points
    yaw = -np.arctan2(y, -x)
    proj_x = 0.5 * (yaw / np.pi + 1.0)  # in [0.0, 1.0]
    new_raw = np.nonzero((proj_x[1:] < 0.2) * (proj_x[:-1] > 0.8))[0] + 1
    proj_y = np.zeros_like(proj_x)
    proj_y[new_raw] = 1
    proj_y = np.cumsum(proj_y)
    # scale to image size using angular resolution
    proj_x = proj_x * proj_w - 0.001
    # --- Wrap around vertical axis if specified ---
    if wrap_around > 0 and wrap_around < proj_w:
        # Shift proj_x so that wrap_around is the new "zero"
        proj_x = (proj_x - wrap_around) % proj_w
        
    # print(f'proj_y: [{proj_y.min()} - {proj_y.max()}] - ({(proj_y < self.proj_h).astype(np.int32).sum()} - {(proj_y >= self.proj_h).astype(np.int32).sum()})')

    # round and clamp for use as index
    proj_x = np.maximum(np.minimum(
        proj_w - 1, np.floor(proj_x)), 0).astype(np.int32)

    proj_y = np.maximum(np.minimum(
        proj_h - 1, np.floor(proj_y)), 0).astype(np.int32)

    # order in decreasing depth
    indices = np.arange(depth.shape[0])
    order = np.argsort(depth)[::-1]
    depth = depth[order]
    indices = indices[order]
    pointcloud = pointcloud[order]
    proj_y = proj_y[order]
    proj_x = proj_x[order]

    # get projection result
    proj_range = np.full((proj_h, proj_w), -1, dtype=np.float32)
    proj_range[proj_y, proj_x] = depth

    proj_pointcloud = np.full((proj_h, proj_w, pointcloud.shape[1]), -1, dtype=np.float32)
    proj_pointcloud[proj_y, proj_x] = pointcloud

    # Concatenate proj_range as the first channel
    proj_range_expanded = proj_range[..., np.newaxis]  # shape (proj_h, proj_w, 1)
    proj_pointcloud = np.concatenate([proj_range_expanded, proj_pointcloud], axis=-1)

    # proj_idx = np.full((proj_h, proj_w), -1, dtype=np.int32)
    # proj_idx[proj_y, proj_x] = indices
    # proj_mask = (proj_idx > 0).astype(np.int32)

    # proj_pointcloud is the 2D numpy array with dimention (proj_h, proj_w, 10) and each elements include range,x,y,z,intensity,flag,r,g,b,label
    # check if proj_pointcloud meet the expected dimention
    assert proj_pointcloud.shape[2] == 10, f"Expected proj_pointcloud to have 10 channels, but got {proj_pointcloud.shape[2]}"

    return proj_pointcloud


In [ ]:
# Read the label data from files
@staticmethod
def readLabel(path):
    label = np.fromfile(path, dtype=np.int32)
    orig_sem_label = label & 0xFFFF  # semantic label in lower half
    # Define the learning map for semantic labels
    # This map is used to convert the original labels to a smaller set of classes
    learning_map = {0: 0, 1: 0, 10: 1, 11: 2, 13: 5, 15: 3, 16: 5, 18: 4, 20: 5,
        30: 6, 31: 7, 32: 8, 40: 9, 44: 10, 48: 11, 49: 12, 50: 13,
        51: 14, 52: 0, 60: 9, 70: 15, 71: 16, 72: 17, 80: 18, 81: 19,
        99: 0, 252: 1, 253: 7, 254: 6, 255: 8, 256: 5, 257: 5, 258: 4, 259: 5}
    # Convert orig_sem_label to sem_label using the map
    sem_label = np.vectorize(learning_map.get)(orig_sem_label)
    return sem_label

In [ ]:
sequences_rootdir = '..\\..\\sequences'
sequences = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

In [ ]:
for seq in tqdm(sequences, desc="Processing Sequences"):
    basedir = os.path.join(sequences_rootdir, seq)
    pointdir = os.path.join(sequences_rootdir, seq, 'velodyne')
    labeldir = os.path.join(basedir, 'labels')
    imgdir = os.path.join(basedir, 'image_2')
    calib_file = os.path.join(basedir, 'calib.txt')
    # search all .bin files in pointdir and put them in a list
    point_files = sorted([f for f in os.listdir(pointdir) if f.endswith('.bin')])
    # search all .png files in imgdir and put them in a list
    img_files = sorted([f for f in os.listdir(imgdir) if f.endswith('.png')])
    # search all .label files in labeldir and put them in a list
    label_files = sorted([f for f in os.listdir(labeldir) if f.endswith('.label')])
    # make sure the number of point files and image files match
    if len(point_files) != len(img_files):
        raise ValueError("Number of point files and image files do not match.")
    if len(point_files) != len(label_files):
        raise ValueError("Number of point files and label files do not match.")

    # save the preprocess point cloud to a new file in the subfolder 'preprocess', use the same filename
    preprocess_dir = os.path.join(basedir, 'preprocess')
    os.makedirs(preprocess_dir, exist_ok=True)

    # go through the list of point files and image files and colorize the points
    for point_file, img_file, label_file in tqdm(zip(point_files, img_files, label_files), total=len(point_files), desc="Colorizing point clouds", leave=False):
        pc_file = os.path.join(pointdir, point_file) 
        im_file = os.path.join(imgdir, img_file)
        label_file = os.path.join(labeldir, label_file)
        # print out the files involved
        # print(f"Processing {pc_file}, {im_file}, {label_file}")
        label = readLabel(label_file)  # N x 1
        points, pts_valid, colors = doColorize(pc_file, im_file, calib_file)
        # check if length of points and labels match
        if len(points) != len(label):
            raise ValueError(f"Number of points ({len(points)}) does not match number of labels ({len(label)}) in file {label_file}.")
        # cascade the labels to the points
        points_output = np.hstack((points, label[:, np.newaxis]))  # add labels
        # do the scan projection
        points_final = doProjection(points_output, wrap_around=1024)
        # save the points to a new file in the preprocess directory
        output_file = os.path.join(preprocess_dir, point_file)
        points_final.astype(np.float32).tofile(output_file)